In [56]:
#script for taking the parameters from blue pebble (BPoptimisedParameters) and sampling the circuits to get results distributions
import sys
import os
import re
import json
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import (
    FakeBrisbane,
    FakeSherbrooke,
    FakeTorino,
)  

In [57]:
#\\\/////////    Functions    //////////
def findFile(directory, identifier):
    identifierLower = identifier.lower()
    try:
        for filename in os.listdir(directory):
            if identifierLower in filename.lower():
                return filename
    except FileNotFoundError:
        print(f"Error: No file found for the folder/identifier combination: {directory} + {identifier}")
        return None
    return None

def load_ising_and_build_hamiltonian(file_path, instance_id):
    """
    Loads Ising terms and weights from a JSON file.
    Determines the number of qubits from the terms and constructs
    the Hamiltonian as a Qiskit SparsePauliOp.
    """

    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  # Assumes this loads a list of dicts

    selected_ising_data = None
    # Find the desired ising model within list
    for ising_instance in all_isings_data:
        if (
            ising_instance["instance_id"] == instance_id
        ):  # Assumes 'instance_id' exists and is correct
            selected_ising_data = ising_instance
            break

    terms = selected_ising_data["terms"]
    weights = selected_ising_data["weights"]
    problem_type = selected_ising_data.get("problem_type")

    pauli_list = []
    num_qubits = 0

    # Find the max number of qubits by finding the biggest index of ising variables
    all_indices = []
    for term_group in terms:
        for idx in term_group:
            all_indices.append(idx)
    num_qubits = max(all_indices) + 1

    for term_indices, weight in zip(terms, weights):
        paulis_arr = ["I"] * num_qubits
        if len(term_indices) == 1:  # Linear term
            paulis_arr[term_indices[0]] = "Z"
        elif len(term_indices) == 2:  # Quadratic term
            paulis_arr[term_indices[0]] = "Z"
            paulis_arr[term_indices[1]] = "Z"

        pauli_list.append(("".join(paulis_arr)[::-1], weight)) # how from_list works here: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp
    hamiltonian = SparsePauliOp.from_list(pauli_list)
    return hamiltonian, num_qubits, problem_type

def save_results_to_json(
    resultsDict, problemClass, qubits, layers, backendName
):
    filename = f"{FILEDIRECTORY}/Sampled_{problemClass}_{qubits}q_p={layers}_{backendName}.json"

    with open(filename, "w") as f:
        json.dump(resultsDict, f, indent=4)

def parse_log_to_dict(directory, identifier):
    targetFile = findFile(directory, identifier)
    print(f"Parsing file: {targetFile}")
    pattern = re.compile(
        r"Instance: (?P<instance>\d+), "
        r"Time: (?P<time>[\d.]+)s, "
        r"Number of optimisation loops: (?P<loops>\d+), "
        r"Cost: (?P<cost>[-.\d]+), " # <-- Added '-' to handle negative costs
        r"Params: \[(?P<params>.*?)\]",
        flags=re.DOTALL # <-- This flag allows '.' to match newlines
    )

    results_dict = {}
    try:
        with open(f"{directory}/{targetFile}", 'r') as f:
            content = f.read()
        # 3. Use re.finditer to find all matches in the entire string
        for match in pattern.finditer(content):
            data = match.groupdict()
            
            # Convert values to the correct data types
            instance_id = int(data['instance']) # <-- Key is lowercase here

            # Parse the space-separated numbers inside the 'Params' string
            param_list = [float(p) for p in data['params'].split()]
            
            # Build the nested dictionary for this instance
            results_dict[instance_id] = {
                "Time": float(data['time']),
                "Number of optimisation loops": int(data['loops']),
                "Cost": float(data['cost']),
                "Params": param_list
            }

    except FileNotFoundError:
        print(f"Error: The file '{targetFile}' was not found.")
        return None

    return results_dict

In [58]:
#//////////// Variables //////////
problemType = "TSP"  # "Knapsack" or "TSP"
samplingInstances = 100 # number of circuits that are formed and sampled aka number of distribution generated
depth = 20 # number of layers in the QAOA circuit
backendSimulator = AerSimulator()
#backendSimulator = AerSimulator.from_backend(FakeTorino())

In [59]:
#//////////// Loading Stuff in //////////
FILEDIRECTORY = "isingBatches"
isingFileName = FILEDIRECTORY+"/batch_Ising_data_TSP_9q_.json"
print(f"Loading Ising data from: {isingFileName}")
parameters = parse_log_to_dict(FILEDIRECTORY, "Params_"+problemType)
print(parameters)

Loading Ising data from: isingBatches/batch_Ising_data_TSP_9q_.json
Parsing file: Optimised_Params_tsp_9q_on_aer_simulator_ideal.txt
{14: {'Time': 12.1098, 'Number of optimisation loops': 234, 'Cost': -10.93310546875, 'Params': [2.17994129, 1.42024479, 0.43010236, 1.78708666, 2.17535797, 2.17168777, 0.66361774, 2.33179308, 0.71930086, 2.37853261, 2.49217662, 0.85780313, 1.79932881, 0.8107207, 2.18672705, 0.22277142, 0.92157732, 1.52944649, 1.30683811, 2.77676042, 0.44182958, 1.76135569, 0.82026979, 2.31974885, 0.51388081, 3.01110482, 0.16869762, 2.09202694, 0.42429945, 2.52565828, 1.88893129, 0.80932203, 2.44606327, 0.86610024, 3.12130919, 0.12563813, 2.19244791, 2.75869893, 1.16520645, 2.49343376]}, 4: {'Time': 12.2459, 'Number of optimisation loops': 241, 'Cost': -12.255859375, 'Params': [2.16020657, 2.36859166, 2.20374517, 1.62443809, 3.24678889, 1.02975157, 1.43257162, 2.04366441, 0.55468995, 3.41205372, 1.97966674, 0.22427043, -0.00586415, 1.45510415, 1.33573712, 1.16496031, 1.613

In [60]:
#///////////// Main Script /////////////
samplingResults = {}
for instanceIndex in range(1, 1 + samplingInstances):
    print(f"Processing instance {instanceIndex}...")
    cost_hamiltonian, num_qubits, problem_type = (
        load_ising_and_build_hamiltonian(isingFileName, instanceIndex)
    )
    if problem_type == "tsp": #including starting state for valid solution for tsp class problems
        initialCircuit = QuantumCircuit(num_qubits)
        initialCircuit.x(
            [0, 4, 8]
        )

    circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=depth, initial_state=initialCircuit)
    circuit.measure_all()
    pm = generate_preset_pass_manager(optimization_level=3, backend=backendSimulator)
    candidate_circuit = pm.run(circuit)
    
    betaGamma = parameters[instanceIndex]["Params"]
    optimized_circuit = candidate_circuit.assign_parameters(betaGamma)
    sampler = Sampler(mode=backendSimulator)
    sampler.options.default_shots = 1000

    result = sampler.run([optimized_circuit]).result()
    dist = result[0].data.meas.get_counts()
    samplingResults[instanceIndex] = dist

save_results_to_json(
    samplingResults, problemType, num_qubits, depth, backendSimulator.name
)


Processing instance 1...
yo mama
Processing instance 2...
yo mama
Processing instance 3...
yo mama
Processing instance 4...
yo mama
Processing instance 5...
yo mama
Processing instance 6...
yo mama
Processing instance 7...
yo mama
Processing instance 8...
yo mama
Processing instance 9...
yo mama
Processing instance 10...
yo mama
Processing instance 11...
yo mama
Processing instance 12...
yo mama
Processing instance 13...
yo mama
Processing instance 14...
yo mama
Processing instance 15...
yo mama
Processing instance 16...
yo mama
Processing instance 17...
yo mama
Processing instance 18...
yo mama
Processing instance 19...
yo mama
Processing instance 20...
yo mama
Processing instance 21...
yo mama
Processing instance 22...
yo mama
Processing instance 23...
yo mama
Processing instance 24...
yo mama
Processing instance 25...
yo mama
Processing instance 26...
yo mama
Processing instance 27...
yo mama
Processing instance 28...
yo mama
Processing instance 29...
yo mama
Processing instance 30.